In [2]:
import requests
from bs4 import BeautifulSoup
import sqlite3
import pandas as pd

In [3]:
# 基本URL（ページ番号部分を除く）
base_url = "https://suumo.jp/jj/chintai/ichiran/FR301FC011/?ar=030&bs=040&fw=%e6%9d%b1%e4%ba%ac%e9%83%bd%e5%8d%83%e4%bb%a3%e7%94%b0%e5%8c%ba&po1=09&page={}"

In [4]:
# 全ページの物件データを保存するリストを初期化
all_properties_list = []

# ページ数の範囲（ここでは1ページ目から3ページ目まで）
for page in range(1, 4):
    # 各ページのURLを生成
    url = base_url.format(page)
    
    # リクエストを送信
    response = requests.get(url)
    
    # BeautifulSoupオブジェクトを作成
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # すべての物件リストアイテムを取得
    property_items = soup.find_all('div', class_='property')

    for item in property_items:
        property_data = {
            '物件名': item.find('a', class_='js-cassetLinkHref').text if item.find('a', class_='js-cassetLinkHref') else '-',
            'URL': 'https://suumo.jp' + item.find('a', class_='js-cassetLinkHref')['href'] if item.find('a', class_='js-cassetLinkHref') else '-',
            'イメージ画像': item.find('img', class_='js-noContextMenu')['rel'] if item.find('img', class_='js-noContextMenu') else '-',
            '賃料': int(float(item.find('div', class_='detailbox-property-point').text.replace('万円', '')) * 10000) if item.find('div', class_='detailbox-property-point') else '-',
            '管理費': item.find('td', class_='detailbox-property--col1').find_all('div')[1].text.replace('管理費 ', '') if item.find('td', class_='detailbox-property--col1') and len(item.find('td', class_='detailbox-property--col1').find_all('div')) > 1 else '-',
            '間取り': item.find('td', class_='detailbox-property--col3').find_all('div')[0].text if item.find('td', class_='detailbox-property--col3') else '-',
            '専有面積': item.find('td', class_='detailbox-property--col3').find_all('div')[1].text.replace('\n', '').strip() if item.find('td', class_='detailbox-property--col3') else '-',
            '向き': item.select('td.detailbox-property--col3')[0].find_all('div')[2].text if len(item.select('td.detailbox-property--col3')[0].find_all('div')) > 2 else '-' if item.select('td.detailbox-property--col3') else '-',
            '物件種別': item.select('td.detailbox-property--col3')[1].find_all('div')[0].text if len(item.select('td.detailbox-property--col3')) > 1 else '-' if item.select('td.detailbox-property--col3') else '-',
            '築年数': item.select('td.detailbox-property--col3')[1].find_all('div')[1].text if len(item.select('td.detailbox-property--col3')[1].find_all('div')) > 1 else '-' if item.select('td.detailbox-property--col3') else '-',
            '住所': item.select_one('td.detailbox-property-col:not([class*="detailbox-property--"])').text.strip() if item.select_one('td.detailbox-property-col:not([class*="detailbox-property--"])') else '-',
            '最寄り駅と距離': item.select_one('.detailnote-box > div').text.strip() if item.select_one('.detailnote-box > div') else '-',
            '敷金': item.find('td', class_='detailbox-property--col2').find_all('div')[0].contents[-1].strip() if item.find('td', class_='detailbox-property--col2') else '-',
            '礼金': item.find('td', class_='detailbox-property--col2').find_all('div')[1].contents[-1].strip() if item.find('td', class_='detailbox-property--col2') else '-',
            '保証金': item.find('div', class_='detailbox-property-inactive', string=lambda text: '保証金' in text).text.replace('保証金', '').strip() if item.find('div', class_='detailbox-property-inactive', string=lambda text: '保証金' in text) else '-',
            '敷引・償却': item.find('div', class_='detailbox-property-inactive', string=lambda text: '敷引・償却' in text).text.replace('敷引・償却', '').strip() if item.find('div', class_='detailbox-property-inactive', string=lambda text: '敷引・償却' in text) else '-',
            '取り扱い店舗': item.select_one('.detailnote-box .detailnote-box-item div:nth-of-type(1)').text.strip() if item.select_one('.detailnote-box .detailnote-box-item div:nth-of-type(1)') else '-',
            '取り扱い店舗の電話番号': item.find('div', class_='ui-text--bold').text.strip() if item.find('div', class_='ui-text--bold') else '-'
        }

        # 各ページの物件データを全体リストに追加
        all_properties_list.append(property_data)

# データフレームに変換
df = pd.DataFrame(all_properties_list)

# データフレームをCSVファイルに出力
df.to_csv('chiyoda_3pages_all_property.csv', index=False, encoding='utf-8-sig')
print("データがCSVファイル 'chiyoda_3pages_all_property.csv' に保存されました。")


データがCSVファイル 'chiyoda_3pages_all_property.csv' に保存されました。
